In [1]:
import pandas as pd
import numpy as np

##Preprocessing

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
anxiety = pd.read_csv("/content/drive/MyDrive/544/Project Files/data/split_Anxiety.csv")
# anxiety

In [12]:
anxiety.columns

Index(['title', 'selftext'], dtype='object')

###With Title

In [5]:
# anxiety = (anxiety["title"] + ". " + anxiety["selftext"]).values

In [6]:
# depression = pd.read_csv("/content/drive/MyDrive/544/Project Files/data/split_depression.csv")

In [7]:
# depression = (depression["title"] + ". " + depression["selftext"]).values

In [8]:
# rndm = pd.read_csv("/content/drive/MyDrive/544/Project Files/data/split_non_depression.csv")

In [9]:
# rndm = (rndm["title"] + ". " + rndm["selftext"]).values

### Without Title

In [13]:
anxiety = (anxiety["selftext"]).values

In [14]:
depression = pd.read_csv("/content/drive/MyDrive/544/Project Files/data/split_depression.csv")

In [15]:
depression = (depression["selftext"]).values

In [16]:
rndm = pd.read_csv("/content/drive/MyDrive/544/Project Files/data/split_non_depression.csv")

In [17]:
rndm = (rndm["selftext"]).values

In [18]:
anxiety.shape, depression.shape, rndm.shape

((1846,), (1827,), (2408,))

In [19]:
anxiety = np.random.choice(anxiety, 1827)
depression = np.random.choice(depression, 1827)
rndm = np.random.choice(rndm, 1827)

In [20]:
x = np.concatenate([depression, rndm, anxiety])
y = np.concatenate([np.ones(len(depression)), np.zeros(len(rndm))])

In [21]:
x.shape, y.shape

((5481,), (3654,))

In [22]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

def metrics(x, y_true, model):
  pred = model.predict(x)
  precision = precision_score(y_true, pred)
  recall = recall_score(y_true, pred)
  f1 = f1_score(y_true, pred)
  print( "Precision: ", precision, ". Recall: ", recall, ". F1: ", f1)



##TFIDF

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

X = vectorizer.fit_transform(x)


In [24]:
anxiety_tfidf = X[len(anxiety)*2:]
X = X[:len(anxiety)*2]

In [25]:
from sklearn.model_selection import train_test_split
train, test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Train

In [30]:
from sklearn.model_selection import KFold
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
def l1SVC_best_paras(X, y):
    best_score = float('-inf')
    best_c = None
    best_model = None
    c_range = np.logspace(start=-3, stop=6, num=10, base=2)

    for c in c_range:
        score = 0
        kf = KFold(n_splits=10)
        for train_index, test_index in kf.split(X):
            cur_X_train, cur_X_val = X[train_index], X[test_index]
            cur_y_train, cur_y_val = y[train_index], y[test_index]

            clf = LinearSVC(penalty='l1', random_state=42, C=c, max_iter=1000, dual=False)
            clf.fit(cur_X_train, cur_y_train)
            score += clf.score(cur_X_val, cur_y_val)
        cur_score = score / 10
        if cur_score > best_score:
            best_model = clf
            best_c = c
    return best_model, best_c

best_svc, best_c = l1SVC_best_paras(train, y_train)
print(best_c)

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  w

64.0


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [31]:

# clf = LinearSVC(max_iter=1000)
clf = best_svc


In [32]:
clf.fit(train, y_train)
metrics(test, y_test, clf)

Precision:  0.9575163398692811 . Recall:  0.9301587301587302 . F1:  0.9436392914653785


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [33]:
metrics(anxiety_tfidf, np.ones(len(anxiety)), clf)

Precision:  1.0 . Recall:  0.6283524904214559 . F1:  0.7717647058823529


In [34]:
from sklearn.linear_model import Perceptron
clf = Perceptron(tol=1e-3, random_state=0)


In [35]:
clf.fit(train, y_train)
metrics(test, y_test, clf)

Precision:  0.9546191247974068 . Recall:  0.9349206349206349 . F1:  0.9446672012830793


In [36]:
metrics(anxiety_tfidf, np.ones(len(anxiety)), clf)

Precision:  1.0 . Recall:  0.7033388067870826 . F1:  0.8258354755784062
